In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
spark = SparkSession.builder.appName("MinConnections").getOrCreate()

23/04/20 23:03:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [18]:
schema = StructType([StructField("id", IntegerType(), True), StructField("name", StringType(), True)])

In [19]:
names = spark.read.schema(schema).option("sep" , " ").csv("Marvel+Names")

In [20]:
lines = spark.read.text("Marvel+Graph")

In [7]:
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

In [8]:
minConnectionCount = connections.agg(func.min("connections")).first()[0]

In [9]:
minConnections = connections.filter(func.col("connections") == minConnectionCount)

In [21]:
minConnectionNames = minConnections.join(names, "id")

In [22]:
print("The following characters have only " + str(minConnectionCount) + " connection(s): ")

The following characters have only 0 connection(s): 


In [23]:
minConnectionNames.select("name").show()

+--------------------+
|                name|
+--------------------+
|        BERSERKER II|
|              BLARE/|
|MARVEL BOY II/MARTIN|
|MARVEL BOY/MARTIN BU|
|      GIURESCU, RADU|
|       CLUMSY FOULUP|
|              FENRIS|
|              RANDAK|
|           SHARKSKIN|
|     CALLAHAN, DANNY|
|         DEATHCHARGE|
|                RUNE|
|         SEA LEOPARD|
|         RED WOLF II|
|              ZANTOR|
|JOHNSON, LYNDON BAIN|
|          LUNATIK II|
|                KULL|
|GERVASE, LADY ALYSSA|
+--------------------+

